In [1]:
# Ignore SQLITE warnings related to Decimal numbers in the Chinook database

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import Dependencies

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, MetaData, Table

### Using SQLAlchemy ORM to reflect and map the tables¶

In [3]:
# Create an engine for the chinook.sqlite database

engine = create_engine("sqlite:///../Resources/chinook.sqlite")

In [4]:
# Reflect Database into ORM classes

Base = automap_base()
Base.prepare(engine,reflect=True)
Base.classes.keys()

['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoice_items',
 'invoices',
 'tracks',
 'media_types',
 'playlists']

In [5]:
# Save a reference to the invoices table as `Invoices`

Invoices = Base.classes.invoices

In [6]:
# Save a reference to the invoice_items table as `Items`

Items = Base.classes.invoice_items

### Use the Metadata and Table object to describe the table and columns.

In [7]:
#  Create a meta data object to hold the reflected table schema

metadata = MetaData()

In [8]:
# Create a table object for  the `invoices` and `invoice_items` tables
# use 'autoload' and 'autoload_with' to define the columns from the table. 

Invoices_table = Table('invoices', metadata, autoload=True, autoload_with=engine)

In [9]:
# Get the column names using the 'column.keys()' methods on the table object. 
InvoiceItem_table = Table('invoice_items', metadata, autoload=True, autoload_with=engine)
print(InvoiceItem_table.columns.keys())

['InvoiceLineId', 'InvoiceId', 'TrackId', 'UnitPrice', 'Quantity']


### Use Python to query the tables. 

In [10]:
# Create a database session object

session = Session(engine)

In [11]:
# List all of the countries found in the invoices table

results = session.query(Invoices.BillingCountry).distinct().all()
for result in results:
    print(result)

('Germany',)
('Norway',)
('Belgium',)
('Canada',)
('USA',)
('France',)
('Ireland',)
('United Kingdom',)
('Australia',)
('Chile',)
('India',)
('Brazil',)
('Portugal',)
('Netherlands',)
('Spain',)
('Sweden',)
('Czech Republic',)
('Finland',)
('Denmark',)
('Italy',)
('Poland',)
('Austria',)
('Hungary',)
('Argentina',)


In [12]:
# Design a query that lists the invoices totals for each billing country 
# and sort the output in descending order.

session.query(Invoices.BillingCountry, func.sum(Invoices.Total)).group_by(Invoices.BillingCountry).order_by().all()

[('Argentina', Decimal('37.62')),
 ('Australia', Decimal('37.62')),
 ('Austria', Decimal('42.62')),
 ('Belgium', Decimal('37.62')),
 ('Brazil', Decimal('190.10')),
 ('Canada', Decimal('303.96')),
 ('Chile', Decimal('46.62')),
 ('Czech Republic', Decimal('90.24')),
 ('Denmark', Decimal('37.62')),
 ('Finland', Decimal('41.62')),
 ('France', Decimal('195.10')),
 ('Germany', Decimal('156.48')),
 ('Hungary', Decimal('45.62')),
 ('India', Decimal('75.26')),
 ('Ireland', Decimal('45.62')),
 ('Italy', Decimal('37.62')),
 ('Netherlands', Decimal('40.62')),
 ('Norway', Decimal('39.62')),
 ('Poland', Decimal('37.62')),
 ('Portugal', Decimal('77.24')),
 ('Spain', Decimal('37.62')),
 ('Sweden', Decimal('38.62')),
 ('USA', Decimal('523.06')),
 ('United Kingdom', Decimal('112.86'))]

In [13]:
# List all of the Billing Postal Codes for the USA.

results = session.query(Invoices.BillingPostalCode).\
    filter(Invoices.BillingCountry == 'USA').group_by(Invoices.BillingPostalCode).all()
results

[('10012-2612',),
 ('2113',),
 ('32801',),
 ('53703',),
 ('60611',),
 ('76110',),
 ('84102',),
 ('85719',),
 ('89503',),
 ('94040-111',),
 ('94043-1351',),
 ('95014',),
 ('98052-8300',)]

In [14]:
# Calculate the Item Totals (sum(UnitPrice * Quantity)) for the USA

session.query(func.sum(Items.Quantity*Items.UnitPrice)).\
where((Invoices.InvoiceId == Items.InvoiceId) & (Invoices.BillingCountry == 'USA')).all()

[(Decimal('523.06'),)]

In [15]:
# Calculate the Item Totals `sum(UnitPrice * Quantity)` for each Billing Postal Code in the USA
# Sort the results in descending order by Total

session.query(func.sum(Items.Quantity*Items.UnitPrice), Invoices.BillingPostalCode)\
.where((Invoices.InvoiceId == Items.InvoiceId) & (Invoices.BillingCountry == 'USA')).group_by(Invoices.BillingPostalCode)\
.order_by(func.sum(Items.Quantity*Items.UnitPrice).desc()).all()

[(Decimal('47.62'), '76110'),
 (Decimal('43.62'), '60611'),
 (Decimal('43.62'), '84102'),
 (Decimal('42.62'), '53703'),
 (Decimal('39.62'), '94040-111'),
 (Decimal('39.62'), '98052-8300'),
 (Decimal('39.62'), '32801'),
 (Decimal('38.62'), '95014'),
 (Decimal('37.62'), '94043-1351'),
 (Decimal('37.62'), '89503'),
 (Decimal('37.62'), '85719'),
 (Decimal('37.62'), '2113'),
 (Decimal('37.62'), '10012-2612')]